In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install dependencies & unzip dataset
!pip install --quiet torch torchvision transformers pillow pandas tqdm scikit-learn
!apt-get install -qq unzip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.9 MB/s eta 0:00:00


In [ ]:
# Train on train/dev
import os, random
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from torch.optim import AdamW

In [ ]:
# 2. Base folder where you unzipped the Hateful Memes files
BASE_DIR   = "/content/drive/MyDrive/data"    # <- your "data" folder under MyDrive
IMAGE_DIR  = os.path.join(BASE_DIR, "img")
TRAIN_FILE = os.path.join(BASE_DIR, "train.jsonl")
DEV_FILE   = os.path.join(BASE_DIR, "dev.jsonl")
TEST_FILE  = os.path.join(BASE_DIR, "test.jsonl")


# 3. Quick sanity check
print("Contents of BASE_DIR:")
!ls -lh "{BASE_DIR}"

print("\nSample images directory:")
!ls -lh "{IMAGE_DIR}" | head -n 10

# Now you can use TRAIN_FILE, DEV_FILE, TEST_FILE, IMAGE_DIR in all subsequent cells.

Contents of BASE_DIR:
total 1.2M
drwx------ 2 root root 4.0K May 23 16:07 blurred_test
drwx------ 2 root root 4.0K May 23 12:14 data
-rw------- 1 root root  54K Jun 14  2020 dev.jsonl
drwx------ 2 root root 4.0K Jun 14  2020 img
-rw------- 1 root root 9.7K Jun 14  2020 LICENSE.txt
-rw------- 1 root root 3.0K Jun 14  2020 README.md
-rw------- 1 root root 100K Jun 14  2020 test.jsonl
-rw------- 1 root root 981K Jun 14  2020 train.jsonl

Sample images directory:
total 3.3G
-rw------- 1 root root  187K Jun 14  2020 01235.png
-rw------- 1 root root  608K Jun 14  2020 01236.png
-rw------- 1 root root  573K Jun 14  2020 01243.png
-rw------- 1 root root  392K Jun 14  2020 01245.png
-rw------- 1 root root   99K Jun 14  2020 01247.png
-rw------- 1 root root  408K Jun 14  2020 01256.png
-rw------- 1 root root  290K Jun 14  2020 01258.png
-rw------- 1 root root  408K Jun 14  2020 01264.png
-rw------- 1 root root  159K Jun 14  2020 01268.png


In [ ]:
!ls -lh "{BASE_DIR}"

total 1.2M
drwx------ 2 root root 4.0K May 23 16:07 blurred_test
drwx------ 2 root root 4.0K May 23 12:14 data
-rw------- 1 root root  54K Jun 14  2020 dev.jsonl
drwx------ 2 root root 4.0K Jun 14  2020 img
-rw------- 1 root root 9.7K Jun 14  2020 LICENSE.txt
-rw------- 1 root root 3.0K Jun 14  2020 README.md
-rw------- 1 root root 100K Jun 14  2020 test.jsonl
-rw------- 1 root root 981K Jun 14  2020 train.jsonl


In [ ]:


# 1. Mount Google Drive (only in Colab)

from google.colab import drive
import os

drive.mount('/content/drive')
BASE_DIR   = "/content/drive/MyDrive/data"
IMAGE_DIR  = os.path.join(BASE_DIR, "img")
TRAIN_FILE = os.path.join(BASE_DIR, "train.jsonl")
# (you can ignore DEV_FILE/TEST_FILE if you only want to train)


# 3. Sanity check
print("Contents of BASE_DIR:")
!ls -lh "{BASE_DIR}"

print("\nSample of IMAGE_DIR:")
!ls -lh "{IMAGE_DIR}" | head -n 10


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of BASE_DIR:
total 1.2M
drwx------ 2 root root 4.0K May 23 16:07 blurred_test
drwx------ 2 root root 4.0K May 23 12:14 data
-rw------- 1 root root  54K Jun 14  2020 dev.jsonl
drwx------ 2 root root 4.0K Jun 14  2020 img
-rw------- 1 root root 9.7K Jun 14  2020 LICENSE.txt
-rw------- 1 root root 3.0K Jun 14  2020 README.md
-rw------- 1 root root 100K Jun 14  2020 test.jsonl
-rw------- 1 root root 981K Jun 14  2020 train.jsonl

Sample of IMAGE_DIR:
total 3.3G
-rw------- 1 root root  187K Jun 14  2020 01235.png
-rw------- 1 root root  608K Jun 14  2020 01236.png
-rw------- 1 root root  573K Jun 14  2020 01243.png
-rw------- 1 root root  392K Jun 14  2020 01245.png
-rw------- 1 root root   99K Jun 14  2020 01247.png
-rw------- 1 root root  408K Jun 14  2020 01256.png
-rw------- 1 root root  290K Jun 14  2020 01258.png
-rw------- 1 root root  408K Jun 14 

In [ ]:
# List everything under your 'data' folder and the first few files inside img/
!find /content/drive/MyDrive/data -maxdepth 2 | sed -e 's/^/FILE: /' | head -n 50


FILE: /content/drive/MyDrive/data
FILE: /content/drive/MyDrive/data/img
FILE: /content/drive/MyDrive/data/img/89573.png
FILE: /content/drive/MyDrive/data/img/90165.png
FILE: /content/drive/MyDrive/data/img/89634.png
FILE: /content/drive/MyDrive/data/img/90243.png
FILE: /content/drive/MyDrive/data/img/90183.png
FILE: /content/drive/MyDrive/data/img/90148.png
FILE: /content/drive/MyDrive/data/img/90125.png
FILE: /content/drive/MyDrive/data/img/90162.png
FILE: /content/drive/MyDrive/data/img/89574.png
FILE: /content/drive/MyDrive/data/img/89610.png
FILE: /content/drive/MyDrive/data/img/89632.png
FILE: /content/drive/MyDrive/data/img/89613.png
FILE: /content/drive/MyDrive/data/img/89743.png
FILE: /content/drive/MyDrive/data/img/89571.png
FILE: /content/drive/MyDrive/data/img/89603.png
FILE: /content/drive/MyDrive/data/img/90214.png
FILE: /content/drive/MyDrive/data/img/89732.png
FILE: /content/drive/MyDrive/data/img/89723.png
FILE: /content/drive/MyDrive/data/img/90186.png
FILE: /content/d

In [ ]:
!cp -r /content/img "/content/drive/MyDrive/data/img/"


cp: cannot stat '/content/img': No such file or directory


In [ ]:
# ─── after you’ve imported everything else, but before you do `model = HateMemesModel()` ─────
from transformers import CLIPModel

class HateMemesModel(nn.Module):
    def __init__(self):
        super().__init__()
        # load the Vision+Text CLIP backbone
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        D = self.clip.config.projection_dim
        # simple linear head on the concatenated image+text embeddings
        self.head = nn.Linear(2 * D, 2)

    def forward(self, input_ids, attention_mask, pixel_values):
        out = self.clip(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            return_dict=True
        )
        # out.image_embeds & out.text_embeds are both shape (B, D)
        feats = torch.cat([out.image_embeds, out.text_embeds], dim=1)  # → (B, 2D)
        return self.head(feats)  # → (B, 2 logits)


In [ ]:
model = HateMemesModel().to(device)


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
# ─── Train on train/dev (with robust image path resolution) ─────────────────────────
import os, random, json
import torch
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from torch.optim import AdamW

# Hyperparams & seed
EPOCHS, BS, LR = 3, 16, 5e-5
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Dataset class
import os, json, torch
from torch.utils.data import Dataset

class HatefulMemesDataset(Dataset):
    def __init__(self, jsonl_path, img_dir, processor, is_test=False):
        # 1) Read all JSONL lines
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            raw = [json.loads(line) for line in f if line.strip()]

        # 2) Filter to only those images that actually exist
        kept = []
        for row in raw:
            img_name = os.path.basename(row['img'])   # e.g. '13459.png'
            img_path = os.path.join(img_dir, img_name)
            if os.path.isfile(img_path):
                kept.append(row)
        dropped = len(raw) - len(kept)
        print(f"Loaded {len(kept)} / {len(raw)} samples (dropped {dropped} missing images).")

        self.samples  = kept
        self.img_dir   = img_dir
        self.processor = processor
        self.is_test   = is_test

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        row = self.samples[idx]
        img_name = os.path.basename(row['img'])
        img_path = os.path.join(self.img_dir, img_name)
        image    = Image.open(img_path).convert("RGB")

        text = row.get('text', "")
        enc = self.processor(
          text=[text],
          images=[image],
          return_tensors="pt",
          padding="max_length",              # pad all sequences to max_length
          truncation=True,                   # cut off anything longer
          max_length=self.processor.tokenizer.model_max_length  # usually 77 for CLIP
)


        out = {
            'input_ids':      enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'pixel_values':   enc['pixel_values'].squeeze(0),
        }
        if self.is_test:
            out['id'] = row['id']
        else:
            out['labels'] = torch.tensor(row['label'], dtype=torch.long)
        return out

# ─── Define collate_fn HERE, before creating DataLoaders ───────
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # collect lists
    input_ids      = [b['input_ids']      for b in batch]
    attention_mask = [b['attention_mask'] for b in batch]
    pixel_values   = [b['pixel_values']   for b in batch]
    labels         = [b['labels']         for b in batch]

    # pad text
    PAD_ID = processor.tokenizer.pad_token_id if hasattr(processor.tokenizer, 'pad_token_id') else 0
    input_ids      = pad_sequence(input_ids,      batch_first=True, padding_value=PAD_ID)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    # stack images and labels
    pixel_values = torch.stack(pixel_values)            # shape: (B,3,H,W)
    labels       = torch.stack(labels)                  # shape: (B,)

    return {
        'input_ids':      input_ids,
        'attention_mask': attention_mask,
        'pixel_values':   pixel_values,
        'labels':         labels
    }
    train_loader = DataLoader(
        train_ds,
        batch_size=BS,
        shuffle=True,
        num_workers=2,
        collate_fn=collate_fn
)
    dev_loader   = DataLoader(
        dev_ds,
        batch_size=BS,
        shuffle=False,
        num_workers=2,
        collate_fn=collate_fn
)

# Model definition

# ─── DataLoaders (only once each!) ─────────────────────────────────
processor   = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
train_ds    = HatefulMemesDataset(TRAIN_FILE, IMAGE_DIR, processor, is_test=False)
dev_ds      = HatefulMemesDataset(DEV_FILE,   IMAGE_DIR, processor, is_test=False)

train_loader = DataLoader(train_ds, batch_size=BS, shuffle=True,
                          num_workers=2, collate_fn=collate_fn)
dev_loader   = DataLoader(dev_ds,   batch_size=BS, shuffle=False,
                          num_workers=2, collate_fn=collate_fn)

# ─── Model & Optimizer ─────────────────────────────────────────────
model = HateMemesModel().to(device)
optim = AdamW(model.parameters(), lr=LR)
os.makedirs("hm_clip_ckpt", exist_ok=True)
best_acc = 0.0

# ─── Training Loop ─────────────────────────────────────────────────
for epoch in range(1, EPOCHS+1):
    # Train
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
        optim.zero_grad()
        logits = model(batch['input_ids'].to(device),
                       batch['attention_mask'].to(device),
                       batch['pixel_values'].to(device))
        loss   = nn.functional.cross_entropy(logits, batch['labels'].to(device))
        loss.backward(); optim.step()
        total_loss += loss.item()

    # Validate
    model.eval()
    preds, gold = [], []
    with torch.no_grad():
        for batch in tqdm(dev_loader, desc=f"Epoch {epoch} Eval"):
            logits = model(batch['input_ids'].to(device),
                           batch['attention_mask'].to(device),
                           batch['pixel_values'].to(device))
            preds += logits.argmax(1).cpu().tolist()
            gold  += batch['labels'].tolist()

    acc = accuracy_score(gold, preds)
    print(f"Epoch {epoch} → Loss: {total_loss/len(train_loader):.4f}, Dev Acc: {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "hm_clip_ckpt/best.pt")
        print("→ Saved new best.pt")

Loaded 8130 / 8500 samples (dropped 370 missing images).
Loaded 479 / 500 samples (dropped 21 missing images).


Epoch 1 Eval: 100%|██████████| 30/30 [00:13<00:00,  2.26it/s]


Epoch 1 → Loss: 0.6326, Dev Acc: 0.5470
→ Saved new best.pt


Epoch 2 Eval: 100%|██████████| 30/30 [00:10<00:00,  2.73it/s]


Epoch 2 → Loss: 0.5913, Dev Acc: 0.5553
→ Saved new best.pt


Epoch 3 Eval: 100%|██████████| 30/30 [00:10<00:00,  2.73it/s]

Epoch 3 → Loss: 0.5762, Dev Acc: 0.5511


In [ ]:
pip install torch transformers opencv-python pillow tkinter


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
def blur_if_hateful(image):
    # …your same CLIP + blur logic…
    return processed_image

iface = gr.Interface(fn=blur_if_hateful,
                     inputs=gr.Image(type="pil"),
                     outputs=gr.Image(type="pil"),
                     title="Offensive Content Blurrer")
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78558db84cc1971a3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2
import torch
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel

# ─── Model Definition ──────────────────────────────────────────────────────────
class HateMemesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        D = self.clip.config.projection_dim
        self.head = nn.Linear(2*D, 2)
    def forward(self, input_ids, attention_mask, pixel_values):
        out = self.clip(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            return_dict=True
        )
        feats = torch.cat([out.image_embeds, out.text_embeds], dim=1)
        return self.head(feats)

# ─── Load model & processor ───────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
model = HateMemesModel().to(device)
# replace with your checkpoint path:
CKPT = "hm_clip_ckpt/best.pt"
model.load_state_dict(torch.load(CKPT, map_location=device))
model.eval()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# ─── GUI Application ──────────────────────────────────────────────────────────
class BlurApp:
    def __init__(self, master):
        self.master = master
        master.title("Offensive Content Blurrer")

        # Buttons
        tk.Button(master, text="Choose Image…", command=self.load_image).pack(pady=5)
        tk.Button(master, text="Process & Blur", command=self.process_image).pack(pady=5)

        # Canvas for images
        self.canvas_in = tk.Canvas(master, width=300, height=300)
        self.canvas_in.pack(side="left", padx=10, pady=10)
        self.canvas_out = tk.Canvas(master, width=300, height=300)
        self.canvas_out.pack(side="right", padx=10, pady=10)

        # Placeholder
        self.img_path = None
        self.tk_img_in = None
        self.tk_img_out = None

    def load_image(self):
        path = filedialog.askopenfilename(
            title="Select image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if not path:
            return
        self.img_path = path
        img = Image.open(path).convert("RGB")
        img.thumbnail((300,300))
        self.tk_img_in = ImageTk.PhotoImage(img)
        self.canvas_in.create_image(150,150, image=self.tk_img_in)
        self.canvas_out.delete("all")

    def process_image(self):
        if not self.img_path:
            messagebox.showwarning("No Image", "Please choose an image first.")
            return
        threading.Thread(target=self._run_inference, daemon=True).start()

    def _run_inference(self):
        # read + preprocess
        img = Image.open(self.img_path).convert("RGB")
        enc = processor(
            text=[""],                  # or prompt the user for text input
            images=[img],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=processor.tokenizer.model_max_length
        ).to(device)
        with torch.no_grad():
            logits = model(
                input_ids=enc["input_ids"],
                attention_mask=enc["attention_mask"],
                pixel_values=enc["pixel_values"]
            )
        hateful = logits.argmax(dim=1).item() == 1

        # blur if hateful
        arr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        if hateful:
            arr = cv2.GaussianBlur(arr, (51,51), 0)
        result = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
        pil_out = Image.fromarray(result)
        pil_out.thumbnail((300,300))
        self.tk_img_out = ImageTk.PhotoImage(pil_out)

        # update UI (on main thread)
        self.master.after(0, self._update_canvas, hateful, pil_out)

    def _update_canvas(self, hateful, pil_out):
        self.canvas_out.create_image(150,150, image=self.tk_img_out)
        status = "Hateful → Blurred" if hateful else "Not Hateful → Unchanged"
        messagebox.showinfo("Result", status)

        # save output
        out_dir = "blurred_output"
        os.makedirs(out_dir, exist_ok=True)
        fname = os.path.basename(self.img_path)
        save_path = os.path.join(out_dir, fname)
        pil_out.save(save_path)
        print(f"Saved result to {save_path}")

# ─── Run the app ───────────────────────────────────────────────────────────────
if __name__ == "__main__":
    root = tk.Tk()
    app = BlurApp(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
!pip install gradio transformers torchvision

import os
import numpy as np
import cv2
from PIL import Image
import torch
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel
import gradio as gr

# ─── load model ─────────────────────────────────────
class HateMemesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        D = self.clip.config.projection_dim
        self.head = nn.Linear(2*D, 2)
    def forward(self, input_ids, attention_mask, pixel_values):
        out = self.clip(input_ids=input_ids,
                        attention_mask=attention_mask,
                        pixel_values=pixel_values,
                        return_dict=True)
        feats = torch.cat([out.image_embeds, out.text_embeds], dim=1)
        return self.head(feats)

device    = "cuda" if torch.cuda.is_available() else "cpu"
model     = HateMemesModel().to(device)
model.load_state_dict(torch.load("hm_clip_ckpt/best.pt", map_location=device))
model.eval()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# ─── inference fn ────────────────────────────────────
def blur_if_hateful(img: Image.Image):
    # preprocess
    enc = processor(text=[""], images=[img], return_tensors="pt",
                    padding="max_length", truncation=True,
                    max_length=processor.tokenizer.model_max_length
                  ).to(device)
    with torch.no_grad():
        logits = model(**enc)
    hateful = logits.argmax(dim=1).item() == 1

    arr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    if hateful:
        arr = cv2.GaussianBlur(arr, (51,51), 0)
    out = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(out)

# ─── Gradio UI ───────────────────────────────────────
demo = gr.Interface(
    fn=blur_if_hateful,
     inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Image(type="pil", label="Processed Image"),
    title="Offensive Content Blurrer",
    description="Detects hateful content with CLIP and blurs if needed."
)

demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://875fb0f839690c8fa5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

# … your imports, model loading, blur_if_hateful() …

# Build your Interface with the new API:
demo = gr.Interface(
    fn=blur_if_hateful,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Image(type="pil", label="Processed Image"),
    title="Offensive Content Blurrer",
    description="Detects hateful content with CLIP and blurs if needed."
)

demo.launch()


NameError: name 'blur_if_hateful' is not defined

In [ ]:
!apt-get update && apt-get install -y tesseract-ocr libtesseract-dev
!pip install pytesseract


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,732 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https:/

In [ ]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel

# ─── your trained model ────────────────────────────────────────────────
class HateMemesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        D = self.clip.config.projection_dim
        self.head = nn.Linear(2*D, 2)
    def forward(self, input_ids, attention_mask, pixel_values):
        out = self.clip(input_ids=input_ids,
                        attention_mask=attention_mask,
                        pixel_values=pixel_values,
                        return_dict=True)
        feats = torch.cat([out.image_embeds, out.text_embeds], dim=1)
        return self.head(feats)

device    = "cuda" if torch.cuda.is_available() else "cpu"
model     = HateMemesModel().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/hm_clip_ckpt/best.pt", map_location=device))
model.eval()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# ─── improved inference / blur function ─────────────────────────────────
def blur_if_hateful(pil_img: Image.Image) -> Image.Image:
    # 1) extract text from the meme
    meme_text = pytesseract.image_to_string(pil_img).strip().replace("\n", " ")
    if not meme_text:
        meme_text = ""   # fallback

    # 2) run CLIP-based classifier
    enc = processor(
        text=[meme_text],
        images=[pil_img],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=processor.tokenizer.model_max_length
    ).to(device)

    with torch.no_grad():
        logits = model(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            pixel_values=enc["pixel_values"],
        )
    probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()
    pred  = int(logits.argmax(dim=-1).item())

    # 3) diagnostic printout
    print(f">>> OCR text: “{meme_text}”")
    print(f">>> logits = {logits.cpu().numpy()}  probs = {probs}  pred = {pred}")

    # 4) blur if “hateful” (class 1)
    arr = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    if pred == 1:
        arr = cv2.GaussianBlur(arr, (51,51), 0)
    out = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(out)

# ─── Gradio demo ─────────────────────────────────────────────────────────
demo = gr.Interface(
    fn=blur_if_hateful,
    inputs=gr.Image(type="pil", label="Upload Meme"),
    outputs=gr.Image(type="pil", label="Processed Image"),
    title="Offensive Content Blurrer",
    description="We OCR the meme caption and run your CLIP‐based classifier.  If it flags hateful, we blur."
)

if __name__=="__main__":
    demo.launch(debug=True)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://09d37d154188e4616f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


>>> OCR text: “mom won tt CCU UT        a”
>>> logits = [[ 0.2945602  -0.33111447]]  probs = [0.65150803 0.34849194]  pred = 0
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://09d37d154188e4616f.gradio.live


In [ ]:
!unzip -o -P KexZs4tn8hujn1nK \
    "/content/drive/MyDrive/hateful_memes.zip" \
    -d "/content/drive/MyDrive/data"


Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/data/data/img/49758.png  
  inflating: /content/drive/MyDrive/data/data/img/49762.png  
  inflating: /content/drive/MyDrive/data/data/img/49785.png  
  inflating: /content/drive/MyDrive/data/data/img/49786.png  
  inflating: /content/drive/MyDrive/data/data/img/49802.png  
  inflating: /content/drive/MyDrive/data/data/img/49805.png  
  inflating: /content/drive/MyDrive/data/data/img/49806.png  
  inflating: /content/drive/MyDrive/data/data/img/49807.png  
  inflating: /content/drive/MyDrive/data/data/img/49810.png  
  inflating: /content/drive/MyDrive/data/data/img/49826.png  
  inflating: /content/drive/MyDrive/data/data/img/49831.png  
  inflating: /content/drive/MyDrive/data/data/img/49832.png  
  inflating: /content/drive/MyDrive/data/data/img/49836.png  
  inflating: /content/drive/MyDrive/data/data/img/49850.png  
  inflating: /content/drive/MyDrive/data/data/img/49856.png  
  inflating: /conte

In [ ]:
BASE_DIR  = "/content/drive/MyDrive/data"
IMAGE_DIR = os.path.join(BASE_DIR, "img")
TEST_FILE = os.path.join(BASE_DIR, "test.jsonl")


In [ ]:
# Inference on test.jsonl (using the local /content/data folder)

import os, json, torch, numpy as np, cv2
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch.nn as nn

# 1) Paths — point at the local /content/data folder
BASE_DIR  = "/content/drive/MyDrive/data/data"   # note the extra /data
IMAGE_DIR = os.path.join(BASE_DIR, "img")
TEST_FILE = os.path.join(BASE_DIR, "test.jsonl")

# sanity check
print("JSONL:", TEST_FILE, "exists?", os.path.exists(TEST_FILE))
print("First few images in:", IMAGE_DIR, ":", os.listdir(IMAGE_DIR)[:5])


CKPT       = "hm_clip_ckpt/best.pt"
OUT_DIR    = "/content/drive/MyDrive/data/blurred_test"
os.makedirs(OUT_DIR, exist_ok=True)

# 2) Model definition (as before)
class HateMemesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        D = self.clip.config.projection_dim
        self.head = nn.Linear(2*D, 2)
    def forward(self, input_ids, attention_mask, pixel_values):
        out   = self.clip(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values,
            return_dict=True
        )
        feats = torch.cat([out.image_embeds, out.text_embeds], dim=1)
        return self.head(feats)

# 3) Load model & processor
device    = "cuda" if torch.cuda.is_available() else "cpu"
model     = HateMemesModel().to(device)
model.load_state_dict(torch.load(CKPT, map_location=device))
model.eval()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 4) Read only test.jsonl
with open(TEST_FILE, 'r') as f:
    test_samples = [json.loads(l) for l in f if l.strip()]

# 5) Inference & blurring
for row in test_samples:
    fname = os.path.basename(row['img'])
    img   = Image.open(os.path.join(IMAGE_DIR, fname)).convert("RGB")

    enc = processor(
        text=[row.get("text","")],
        images=[img],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=processor.tokenizer.model_max_length
    ).to(device)

    with torch.no_grad():
        logits = model(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            pixel_values=enc["pixel_values"]
        )
    is_hateful = logits.argmax(dim=1).item() == 1

    arr     = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    out_img = cv2.GaussianBlur(arr, (51,51), 0) if is_hateful else arr
    cv2.imwrite(os.path.join(OUT_DIR, fname), out_img)

print(f" Done! Blurred images in `{OUT_DIR}/`")


JSONL: /content/drive/MyDrive/data/data/test.jsonl exists? True
First few images in: /content/drive/MyDrive/data/data/img : ['01235.png', '01236.png', '01243.png', '01245.png', '01247.png']
 Done! Blurred images in `/content/drive/MyDrive/data/blurred_test/`


In [ ]:
# Search for test.jsonl anywhere under /content
!find /content -type f | grep test.jsonl


/content/drive/MyDrive/data/test.jsonl
/content/drive/MyDrive/data/data/test.jsonl


In [ ]:
# List a few of the formerly-missing files:
!ls /content/drive/MyDrive/data/img/12785.png
!ls /content/drive/MyDrive/data/img/13459.png


ls: cannot access '/content/drive/MyDrive/data/img/12785.png': No such file or directory
ls: cannot access '/content/drive/MyDrive/data/img/13459.png': No such file or directory
